In [1]:
import os
import chainlit as cl
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

model_id = "openchat/openchat_3.5"
conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature": 0.8, "max_new_tokens": 150})

template = """You are a helpful AI assistant providing support to someone dealing with phobias. Your goal is to offer friendly and informative advice on overcoming fears. 

{query}
"""

@cl.on_chat_start
def main():
    prompt = PromptTemplate(template=template, input_variables=['query'])
    conv_chain = LLMChain(llm=conv_model, prompt=prompt, verbose=True)
    cl.user_session.set("llm_chain", conv_chain)

@cl.on_message
async def process_message(message: str):
    llm_chain = cl.user_session.get("llm_chain")
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])

    # Perform post-processing on the received response here
    # 'res' is a dictionary, and the response text is stored under the key "text"
    return res["text"]

# If you are using Flask
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Server is running."

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8080)


ModuleNotFoundError: No module named 'chainlit'

In [ ]:
%run Chatbot.ipynb